# Week 7: Localization and Mapping

## Module II: Perception & Localization

### Topics Covered

- Global Navigation Satellite Systems (GNSS/GPS)
- Inertial Measurement Units (IMU) and Dead Reckoning
- Particle Filters for Global Localization (Monte Carlo Localization)
- Simultaneous Localization and Mapping (SLAM)
- High-Definition (HD) Maps and Map Matching
- Multi-Sensor Fusion for Robust Localization

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand GPS/GNSS principles, error sources, and accuracy levels
2. Implement IMU-based dead reckoning and understand drift accumulation
3. Build a complete Particle Filter for global localization with resampling
4. Implement EKF-SLAM to simultaneously build a map and localize
5. Use HD maps for precise lane-level localization via map matching
6. Design multi-sensor fusion architectures for robust localization
7. Evaluate localization accuracy and understand failure modes

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Ellipse
from matplotlib.animation import FuncAnimation
from scipy.stats import multivariate_normal
from collections import defaultdict

# Set random seed for reproducibility
np.random.seed(42)

## 1. Global Navigation Satellite Systems (GNSS/GPS)

GNSS provides global position estimates by triangulating signals from satellites. GPS is the most widely known GNSS operated by the United States.

### How GPS Works

**Trilateration Principle:**
- Satellites continuously broadcast their position and time
- GPS receiver measures time delay to calculate distance to each satellite
- Need at least 4 satellites for 3D position (x, y, z) + clock bias

**Position Calculation:**

Distance to satellite $i$:
$$\rho_i = c \cdot \Delta t_i$$

Where:
- $c$ = speed of light (≈ 3 × 10⁸ m/s)
- $\Delta t_i$ = time delay from satellite $i$

**Trilateration Equations:**
$$\sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} = \rho_i$$

For each satellite $i \in \{1, 2, 3, 4\}$.

### Error Sources

1. **Atmospheric Delays**
   - Ionospheric delay: ±5-10 meters
   - Tropospheric delay: ±0.5-5 meters

2. **Multipath Effects**
   - Signal reflections from buildings/terrain
   - Urban canyons: ±10-50 meters

3. **Satellite Geometry (DOP - Dilution of Precision)**
   - Poor geometry → large errors
   - GDOP (Geometric DOP), HDOP (Horizontal), VDOP (Vertical)

4. **Clock Errors**
   - Satellite clock drift
   - Receiver clock bias

5. **Selective Availability** (historical, now disabled)

### GPS Accuracy Levels

| System | Accuracy | Notes |
|--------|----------|-------|
| Standard GPS | ±5-10m | Consumer devices |
| DGPS (Differential) | ±1-3m | Ground-based corrections |
| RTK GPS | ±2cm | Real-time kinematic, requires base station |
| PPP (Precise Point) | ±10cm | Post-processing |

### Limitations for Autonomous Vehicles

- **Update Rate**: 1-10 Hz (too slow for high-speed control)
- **Latency**: 100-200ms
- **Availability**: Poor in tunnels, parking garages, urban canyons
- **Accuracy**: Insufficient for lane-level localization (need ±10cm)

**Solution**: Fuse GPS with other sensors (IMU, odometry, cameras, LIDAR) using Kalman Filters or Particle Filters.

In [ ]:
# Simple HD Map representation and localization simulation

class HDMap:
    def __init__(self):
        """Simple HD Map with lane boundaries and landmarks"""
        # Create a curved road - compute all points first
        s_values = np.linspace(0, 50, 100)
        
        # Centerline (curved path)
        x_values = s_values
        y_values = 5 * np.sin(s_values / 10)
        
        # Compute heading from gradient
        dx = np.gradient(x_values)
        dy = np.gradient(y_values)
        headings = np.arctan2(dy, dx)
        
        # Lane boundaries (offset from centerline)
        lane_width = 3.5
        
        # Left boundary
        x_left = x_values - lane_width/2 * np.sin(headings)
        y_left = y_values + lane_width/2 * np.cos(headings)
        
        # Right boundary
        x_right = x_values + lane_width/2 * np.sin(headings)
        y_right = y_values - lane_width/2 * np.cos(headings)
        
        self.road_centerline = np.column_stack([x_values, y_values])
        self.lane_left = np.column_stack([x_left, y_left])
        self.lane_right = np.column_stack([x_right, y_right])
        
        # Add some landmark features (poles, signs)
        self.landmarks = np.array([
            [10, 8],
            [20, 7],
            [30, 8],
            [40, 7]
        ])
    
    def get_closest_point_on_centerline(self, position):
        """Find closest point on road centerline"""
        distances = np.linalg.norm(self.road_centerline - position, axis=1)
        idx = np.argmin(distances)
        return self.road_centerline[idx], idx
    
    def visualize(self, ax):
        """Draw HD map on axis"""
        ax.plot(self.road_centerline[:, 0], self.road_centerline[:, 1], 
               'k--', linewidth=1, label='Centerline')
        ax.plot(self.lane_left[:, 0], self.lane_left[:, 1], 
               'y-', linewidth=2, label='Lane Boundaries')
        ax.plot(self.lane_right[:, 0], self.lane_right[:, 1], 'y-', linewidth=2)
        ax.scatter(self.landmarks[:, 0], self.landmarks[:, 1], 
                  c='red', s=100, marker='^', label='Landmarks', zorder=10)


def demonstrate_hd_map_localization():
    """Demonstrate localization using HD map matching"""
    
    # Create HD map
    hd_map = HDMap()
    
    # Simulate vehicle trajectory (with GPS noise)
    true_trajectory = []
    noisy_gps_trajectory = []
    
    for i in range(0, 80, 2):
        # True position (on centerline)
        true_pos = hd_map.road_centerline[i]
        true_trajectory.append(true_pos)
        
        # Noisy GPS
        gps_pos = true_pos + np.random.randn(2) * 3  # ±3m GPS noise
        noisy_gps_trajectory.append(gps_pos)
    
    true_trajectory = np.array(true_trajectory)
    noisy_gps_trajectory = np.array(noisy_gps_trajectory)
    
    # Map-matched trajectory (project GPS onto road)
    map_matched_trajectory = []
    
    for gps_pos in noisy_gps_trajectory:
        closest_point, _ = hd_map.get_closest_point_on_centerline(gps_pos)
        map_matched_trajectory.append(closest_point)
    
    map_matched_trajectory = np.array(map_matched_trajectory)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left: Map with trajectories
    ax = axes[0]
    hd_map.visualize(ax)
    
    ax.plot(true_trajectory[:, 0], true_trajectory[:, 1], 
           'g-', linewidth=3, label='True Position', zorder=5)
    ax.scatter(noisy_gps_trajectory[:, 0], noisy_gps_trajectory[:, 1], 
              c='red', s=30, alpha=0.5, label='Noisy GPS', zorder=3)
    ax.plot(map_matched_trajectory[:, 0], map_matched_trajectory[:, 1], 
           'b-', linewidth=2, label='Map-Matched', zorder=4)
    
    ax.set_xlabel('X Position (m)')
    ax.set_ylabel('Y Position (m)')
    ax.set_title('HD Map Localization')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    
    # Right: Error comparison
    ax2 = axes[1]
    
    gps_error = np.linalg.norm(noisy_gps_trajectory - true_trajectory, axis=1)
    map_matched_error = np.linalg.norm(map_matched_trajectory - true_trajectory, axis=1)
    
    steps = np.arange(len(gps_error))
    ax2.plot(steps, gps_error, 'r-', linewidth=2, label='GPS Error', alpha=0.7)
    ax2.plot(steps, map_matched_error, 'b-', linewidth=2, label='Map-Matched Error')
    ax2.axhline(np.mean(gps_error), color='r', linestyle='--', alpha=0.5,
               label=f'Mean GPS: {np.mean(gps_error):.2f}m')
    ax2.axhline(np.mean(map_matched_error), color='b', linestyle='--', alpha=0.5,
               label=f'Mean Matched: {np.mean(map_matched_error):.2f}m')
    
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('Position Error (m)')
    ax2.set_title('Localization Accuracy Comparison')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"GPS Mean Error: {np.mean(gps_error):.2f} m")
    print(f"Map-Matched Mean Error: {np.mean(map_matched_error):.2f} m")
    print(f"Improvement: {(1 - np.mean(map_matched_error)/np.mean(gps_error))*100:.1f}%")

demonstrate_hd_map_localization()

In [ ]:
# Complete Particle Filter Implementation

class ParticleFilter:
    """Monte Carlo Localization with landmark-based sensing."""
    
    def __init__(self, n_particles, map_limits, landmarks):
        """
        Args:
            n_particles: Number of particles
            map_limits: [x_min, x_max, y_min, y_max]
            landmarks: Nx2 array of landmark positions
        """
        self.n_particles = n_particles
        self.map_limits = map_limits
        self.landmarks = landmarks
        
        # Initialize particles uniformly over map (global localization)
        self.particles = self._initialize_particles()
        self.weights = np.ones(n_particles) / n_particles
        
        # Motion model noise
        self.motion_noise_std = np.array([0.1, 0.1, 0.05])  # [x, y, theta]
        
        # Sensor model noise
        self.sensor_noise_std = np.array([0.5, 0.1])  # [range, bearing]
        
    def _initialize_particles(self):
        """Initialize particles uniformly over the map."""
        x_min, x_max, y_min, y_max = self.map_limits
        
        particles = np.zeros((self.n_particles, 3))  # [x, y, theta]
        particles[:, 0] = np.random.uniform(x_min, x_max, self.n_particles)
        particles[:, 1] = np.random.uniform(y_min, y_max, self.n_particles)
        particles[:, 2] = np.random.uniform(-np.pi, np.pi, self.n_particles)
        
        return particles
    
    def predict(self, control):
        """
        Motion update: apply control with noise to each particle.
        
        Args:
            control: [dx, dy, dtheta] - motion command
        """
        # Add noise to control
        noise = np.random.randn(self.n_particles, 3) * self.motion_noise_std
        
        # Update particles
        self.particles[:, 0] += control[0] + noise[:, 0]
        self.particles[:, 1] += control[1] + noise[:, 1]
        self.particles[:, 2] += control[2] + noise[:, 2]
        
        # Normalize angles
        self.particles[:, 2] = np.arctan2(np.sin(self.particles[:, 2]), 
                                           np.cos(self.particles[:, 2]))
    
    def update(self, measurement):
        """
        Measurement update: compute weights based on sensor likelihood.
        
        Args:
            measurement: [range, bearing] to a landmark
        """
        # For each particle, compute expected measurement to closest landmark
        for i in range(self.n_particles):
            x, y, theta = self.particles[i]
            
            # Find closest landmark
            distances = np.linalg.norm(self.landmarks - np.array([x, y]), axis=1)
            closest_idx = np.argmin(distances)
            lx, ly = self.landmarks[closest_idx]
            
            # Expected measurement
            expected_range = np.sqrt((lx - x)**2 + (ly - y)**2)
            expected_bearing = np.arctan2(ly - y, lx - x) - theta
            expected_bearing = np.arctan2(np.sin(expected_bearing), 
                                           np.cos(expected_bearing))
            
            # Measurement likelihood (Gaussian)
            range_error = measurement[0] - expected_range
            bearing_error = measurement[1] - expected_bearing
            bearing_error = np.arctan2(np.sin(bearing_error), 
                                        np.cos(bearing_error))
            
            # Compute weight (likelihood)
            weight = np.exp(-0.5 * (
                (range_error / self.sensor_noise_std[0])**2 +
                (bearing_error / self.sensor_noise_std[1])**2
            ))
            
            self.weights[i] = weight
        
        # Normalize weights
        self.weights += 1e-300  # Avoid divide by zero
        self.weights /= np.sum(self.weights)
    
    def resample(self):
        """Low-variance resampling."""
        N = self.n_particles
        new_particles = []
        
        # Cumulative sum
        c = np.cumsum(self.weights)
        
        # Start at random position
        u = np.random.uniform(0, 1/N)
        
        i = 0
        for j in range(N):
            # Evenly spaced samples
            threshold = u + j / N
            
            # Find particle
            while threshold > c[i]:
                i += 1
            
            new_particles.append(self.particles[i].copy())
        
        self.particles = np.array(new_particles)
        self.weights = np.ones(N) / N
    
    def effective_sample_size(self):
        """Compute effective sample size."""
        return 1.0 / np.sum(self.weights ** 2)
    
    def estimate(self):
        """Return weighted mean of particles."""
        mean_x = np.sum(self.weights * self.particles[:, 0])
        mean_y = np.sum(self.weights * self.particles[:, 1])
        
        # Circular mean for angle
        sin_sum = np.sum(self.weights * np.sin(self.particles[:, 2]))
        cos_sum = np.sum(self.weights * np.cos(self.particles[:, 2]))
        mean_theta = np.arctan2(sin_sum, cos_sum)
        
        return np.array([mean_x, mean_y, mean_theta])


# Simulation: Robot moving in environment with landmarks

np.random.seed(42)

# Environment setup
map_limits = [0, 50, 0, 50]
landmarks = np.array([
    [10, 10],
    [40, 10],
    [10, 40],
    [40, 40],
    [25, 25]
])

# True robot trajectory (circular path)
n_steps = 50
true_trajectory = []
estimated_trajectory = []

# Initial true position
true_pos = np.array([25.0, 15.0, 0.0])

# Create particle filter
pf = ParticleFilter(n_particles=300, map_limits=map_limits, landmarks=landmarks)

# Simulate
for step in range(n_steps):
    # True motion (circular path)
    velocity = 1.0
    angular_vel = 0.15
    dt = 0.5
    
    control = np.array([
        velocity * np.cos(true_pos[2]) * dt,
        velocity * np.sin(true_pos[2]) * dt,
        angular_vel * dt
    ])
    
    # Update true position
    true_pos += control
    true_pos[2] = np.arctan2(np.sin(true_pos[2]), np.cos(true_pos[2]))
    true_trajectory.append(true_pos.copy())
    
    # Particle filter prediction
    pf.predict(control)
    
    # Generate measurement (to closest landmark from true position)
    distances = np.linalg.norm(landmarks - true_pos[:2], axis=1)
    closest_idx = np.argmin(distances)
    lx, ly = landmarks[closest_idx]
    
    true_range = np.sqrt((lx - true_pos[0])**2 + (ly - true_pos[1])**2)
    true_bearing = np.arctan2(ly - true_pos[1], lx - true_pos[0]) - true_pos[2]
    true_bearing = np.arctan2(np.sin(true_bearing), np.cos(true_bearing))
    
    # Add measurement noise
    measurement = np.array([
        true_range + np.random.randn() * 0.5,
        true_bearing + np.random.randn() * 0.1
    ])
    
    # Particle filter update
    pf.update(measurement)
    
    # Resample if needed
    if pf.effective_sample_size() < pf.n_particles / 2:
        pf.resample()
    
    # Get estimate
    estimate = pf.estimate()
    estimated_trajectory.append(estimate.copy())

true_trajectory = np.array(true_trajectory)
estimated_trajectory = np.array(estimated_trajectory)

# Visualization
fig = plt.figure(figsize=(18, 6))

# Plot 1: Full trajectory with particles (final step)
ax1 = fig.add_subplot(1, 3, 1)

# Draw landmarks
ax1.scatter(landmarks[:, 0], landmarks[:, 1], c='red', s=200, marker='^', 
            edgecolors='black', linewidths=2, label='Landmarks', zorder=10)

# Draw particles (final step)
ax1.scatter(pf.particles[:, 0], pf.particles[:, 1], c='blue', s=10, alpha=0.3, 
            label=f'Particles (N={pf.n_particles})')

# Draw trajectories
ax1.plot(true_trajectory[:, 0], true_trajectory[:, 1], 'g-', 
         linewidth=3, label='True Trajectory', zorder=5)
ax1.plot(estimated_trajectory[:, 0], estimated_trajectory[:, 1], 'r--', 
         linewidth=2, label='PF Estimate', zorder=4)

# Mark start and end
ax1.scatter([true_trajectory[0, 0]], [true_trajectory[0, 1]], 
            c='green', s=300, marker='o', edgecolors='black', linewidths=2, 
            label='Start', zorder=11)
ax1.scatter([true_trajectory[-1, 0]], [true_trajectory[-1, 1]], 
            c='orange', s=300, marker='s', edgecolors='black', linewidths=2, 
            label='End', zorder=11)

ax1.set_xlabel('X (m)', fontsize=12)
ax1.set_ylabel('Y (m)', fontsize=12)
ax1.set_title('Particle Filter Localization', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(map_limits[:2])
ax1.set_ylim(map_limits[2:])
ax1.set_aspect('equal')

# Plot 2: Error over time
ax2 = fig.add_subplot(1, 3, 2)

error = np.linalg.norm(estimated_trajectory[:, :2] - true_trajectory[:, :2], axis=1)
steps = np.arange(len(error))

ax2.plot(steps, error, 'b-', linewidth=2)
ax2.axhline(np.mean(error), color='r', linestyle='--', linewidth=2, 
            label=f'Mean: {np.mean(error):.2f}m')
ax2.fill_between(steps, 0, error, alpha=0.3)

ax2.set_xlabel('Time Step', fontsize=12)
ax2.set_ylabel('Position Error (m)', fontsize=12)
ax2.set_title('Localization Accuracy', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

# Plot 3: Particle convergence visualization (showing 5 snapshots)
ax3 = fig.add_subplot(1, 3, 3)

snapshot_steps = [0, 10, 20, 30, n_steps-1]
colors = plt.cm.viridis(np.linspace(0, 1, len(snapshot_steps)))

# Draw landmarks
ax3.scatter(landmarks[:, 0], landmarks[:, 1], c='red', s=150, marker='^', 
            edgecolors='black', linewidths=1.5, zorder=10)

# Draw true trajectory
ax3.plot(true_trajectory[:, 0], true_trajectory[:, 1], 'g-', 
         linewidth=2, alpha=0.5, label='True Path', zorder=5)

# Particle spread at different times (we can't show historical particles, 
# so let's just show error bars)
for i, step_idx in enumerate(snapshot_steps):
    if step_idx < len(estimated_trajectory):
        ax3.scatter([estimated_trajectory[step_idx, 0]], 
                   [estimated_trajectory[step_idx, 1]], 
                   c=[colors[i]], s=100, marker='o', edgecolors='black',
                   label=f'Step {step_idx}', zorder=6)

ax3.set_xlabel('X (m)', fontsize=12)
ax3.set_ylabel('Y (m)', fontsize=12)
ax3.set_title('Particle Filter Convergence', fontsize=14, fontweight='bold')
ax3.legend(fontsize=9, loc='upper right')
ax3.grid(True, alpha=0.3)
ax3.set_xlim(map_limits[:2])
ax3.set_ylim(map_limits[2:])
ax3.set_aspect('equal')

plt.tight_layout()
plt.show()

print("=" * 70)
print("PARTICLE FILTER LOCALIZATION RESULTS")
print("=" * 70)
print(f"Number of particles: {pf.n_particles}")
print(f"Number of landmarks: {len(landmarks)}")
print(f"Number of time steps: {n_steps}")
print(f"\nLocalization Error:")
print(f"  Mean error: {np.mean(error):.3f} m")
print(f"  Max error: {np.max(error):.3f} m")
print(f"  Min error: {np.min(error):.3f} m")
print(f"  Final error: {error[-1]:.3f} m")
print(f"\nFinal Effective Sample Size: {pf.effective_sample_size():.1f}")
print(f"Resampling threshold: {pf.n_particles / 2:.1f}")
print("=" * 70)

## 3. Particle Filter for Global Localization

**Particle Filter** (Monte Carlo Localization) is a non-parametric Bayes filter that represents the belief as a set of weighted particles.

### Why Particle Filters?

**Advantages over Kalman Filters**:
- ✅ Can represent **multi-modal** distributions (multiple hypotheses)
- ✅ No Gaussian assumption (works with arbitrary sensor models)
- ✅ Can handle **global localization** (unknown initial position)
- ✅ Naturally handles **non-linear** motion and sensor models

**Disadvantages**:
- ❌ Computationally expensive (need 100s-1000s of particles)
- ❌ Can suffer from **particle deprivation** (all particles have low weight)
- ❌ Requires careful tuning of noise parameters

---

### Algorithm Overview

**Representation**: Belief is a set of N particles:
$$\mathcal{X}_t = \{x_t^{[1]}, x_t^{[2]}, ..., x_t^{[N]}\}$$

Each particle $x_t^{[i]}$ represents a hypothesis of the robot's pose: $[x, y, \theta]$.

**Algorithm** (4 steps):

1. **Prediction** (Motion Update):
   - For each particle, apply motion model with noise
   - $x_t^{[i]} \sim p(x_t | u_t, x_{t-1}^{[i]})$

2. **Measurement Update** (Weight Update):
   - Compute importance weight for each particle
   - $w_t^{[i]} = p(z_t | x_t^{[i]})$
   - Higher weight if sensor measurement matches particle's expected measurement

3. **Normalization**:
   - Normalize weights: $w_t^{[i]} = \frac{w_t^{[i]}}{\sum_j w_t^{[j]}}$

4. **Resampling**:
   - Draw N new particles from old particles with probability proportional to weight
   - Prevents particle degeneracy

---

### Motion Model

**Odometry-based** (most common for wheeled robots):

Given control $u_t = [\Delta x, \Delta y, \Delta \theta]$:

$$x_t^{[i]} = x_{t-1}^{[i]} + \Delta x + \mathcal{N}(0, \sigma_x^2)$$
$$y_t^{[i]} = y_{t-1}^{[i]} + \Delta y + \mathcal{N}(0, \sigma_y^2)$$
$$\theta_t^{[i]} = \theta_{t-1}^{[i]} + \Delta \theta + \mathcal{N}(0, \sigma_\theta^2)$$

Noise $\sigma$ captures uncertainty in motion.

---

### Sensor Model

**Landmark-based** (common for localization):

Measure distance and bearing to known landmarks:
$$z = [r, \phi]$$

Where:
- $r$ = distance to landmark
- $\phi$ = bearing (angle) to landmark

**Expected measurement** for particle $x^{[i]} = [x, y, \theta]$:

$$\hat{r} = \sqrt{(x_L - x)^2 + (y_L - y)^2}$$
$$\hat{\phi} = \text{atan2}(y_L - y, x_L - x) - \theta$$

**Likelihood** (probability of measurement given particle):

$$p(z | x^{[i]}) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(r - \hat{r})^2}{2\sigma_r^2} - \frac{(\phi - \hat{\phi})^2}{2\sigma_\phi^2}\right)$$

---

### Resampling

**Why**: After many iterations, few particles have significant weight → inefficient.

**Effective Sample Size** (ESS):
$$N_{eff} = \frac{1}{\sum_{i=1}^{N} (w^{[i]})^2}$$

**Resample when**: $N_{eff} < N_{threshold}$ (e.g., N/2).

**Methods**:

1. **Multinomial Resampling** (simple):
   - Draw N samples with replacement from $\mathcal{X}_t$ with probability $w^{[i]}$

2. **Systematic Resampling** (better):
   - Deterministic, low variance
   - Single random number, evenly spaced samples

3. **Stratified Resampling**

**Low Variance Resampling** (Systematic):
```python
def low_variance_resample(particles, weights):
    N = len(particles)
    new_particles = []
    
    # Cumulative sum
    c = np.cumsum(weights)
    
    # Start at random position
    u = np.random.uniform(0, 1/N)
    
    i = 0
    for j in range(N):
        # Evenly spaced samples
        threshold = u + j / N
        
        # Find particle
        while threshold > c[i]:
            i += 1
        
        new_particles.append(particles[i].copy())
    
    return new_particles
```

---

### Kidnapped Robot Problem

**Challenge**: Robot is "kidnapped" (teleported to unknown location).

**Solution**: **Random particle injection**
- Occasionally inject uniform random particles (5-10%)
- Helps recover from catastrophic failures
- Trade-off: Reduces tracking accuracy

---

### Convergence

**Global Localization**: Start with uniform distribution over entire map.

**Convergence criteria**:
- Particles converge to a small region
- Low variance in particle distribution
- High maximum weight

**Failure modes**:
- **Particle deprivation**: All particles far from true pose
- **Symmetry**: Multiple similar locations (e.g., identical hallways)

---

### Computational Complexity

- **Prediction**: $O(N)$ - apply motion model to N particles
- **Update**: $O(N \cdot M)$ - N particles, M landmarks
- **Resampling**: $O(N)$

**Total**: $O(N \cdot M)$ per time step.

**Typical values**: N = 100-1000 particles, runs at 10-30 Hz on modern CPUs.

In [ ]:
# IMU Dead Reckoning Simulation

from dataclasses import dataclass

@dataclass
class IMUReading:
    """IMU sensor reading."""
    acceleration: np.ndarray  # [ax, ay] in m/s²
    angular_velocity: float   # omega in rad/s
    timestamp: float

class IMUDeadReckoning:
    """Simple 2D IMU-based dead reckoning."""
    
    def __init__(self, initial_pose, acc_noise_std=0.02, gyro_noise_std=0.01):
        """
        Args:
            initial_pose: [x, y, theta]
            acc_noise_std: Accelerometer noise (m/s²)
            gyro_noise_std: Gyroscope noise (rad/s)
        """
        self.pose = np.array(initial_pose, dtype=float)  # [x, y, theta]
        self.velocity = np.array([0.0, 0.0])  # [vx, vy]
        
        self.acc_noise_std = acc_noise_std
        self.gyro_noise_std = gyro_noise_std
        
        # Sensor biases (constant drift)
        self.acc_bias = np.array([0.005, 0.003])  # Small constant bias
        self.gyro_bias = 0.002  # rad/s
        
    def update(self, imu_reading, dt):
        """
        Update pose using IMU reading.
        
        Args:
            imu_reading: IMUReading object
            dt: Time step (seconds)
        """
        # Add noise and bias to measurements
        acc_measured = imu_reading.acceleration + self.acc_bias + \
                       np.random.randn(2) * self.acc_noise_std
        gyro_measured = imu_reading.angular_velocity + self.gyro_bias + \
                        np.random.randn() * self.gyro_noise_std
        
        # Rotate acceleration to global frame
        theta = self.pose[2]
        R = np.array([[np.cos(theta), -np.sin(theta)],
                      [np.sin(theta), np.cos(theta)]])
        acc_global = R @ acc_measured
        
        # Update velocity
        self.velocity += acc_global * dt
        
        # Update position
        self.pose[0] += self.velocity[0] * dt + 0.5 * acc_global[0] * dt**2
        self.pose[1] += self.velocity[1] * dt + 0.5 * acc_global[1] * dt**2
        
        # Update orientation
        self.pose[2] += gyro_measured * dt
        
        # Normalize angle to [-pi, pi]
        self.pose[2] = np.arctan2(np.sin(self.pose[2]), np.cos(self.pose[2]))
        
        return self.pose.copy()

# Simulate vehicle motion with IMU
np.random.seed(42)

# Ground truth trajectory: circular motion
dt = 0.1  # 10 Hz update rate
T = 20    # 20 seconds
n_steps = int(T / dt)

# True vehicle motion
true_positions = []
imu_positions = []
gps_positions = []

# Initial state
true_pos = np.array([0.0, 0.0, 0.0])  # [x, y, theta]
true_vel = 2.0  # m/s
angular_vel = 0.3  # rad/s (turning)

# Initialize IMU dead reckoning
imu_dr = IMUDeadReckoning(initial_pose=true_pos.copy())

# GPS parameters
gps_update_rate = 1.0  # 1 Hz
gps_noise_std = 2.0    # 2 meter standard deviation

for i in range(n_steps):
    t = i * dt
    
    # True motion (circular path)
    true_pos[2] += angular_vel * dt  # Update heading
    true_pos[0] += true_vel * np.cos(true_pos[2]) * dt
    true_pos[1] += true_vel * np.sin(true_pos[2]) * dt
    true_positions.append(true_pos[:2].copy())
    
    # Generate IMU reading (true acceleration in body frame)
    # Centripetal acceleration for circular motion
    a_tangential = 0.0  # Constant speed
    a_centripetal = true_vel * angular_vel  # v * omega
    
    # In body frame: forward = tangential, left = centripetal
    acc_body = np.array([a_tangential, a_centripetal])
    
    imu_reading = IMUReading(
        acceleration=acc_body,
        angular_velocity=angular_vel,
        timestamp=t
    )
    
    # Update IMU dead reckoning
    imu_pos = imu_dr.update(imu_reading, dt)
    imu_positions.append(imu_pos[:2].copy())
    
    # GPS update (slower rate)
    if i % int(gps_update_rate / dt) == 0:
        gps_pos = true_pos[:2] + np.random.randn(2) * gps_noise_std
        gps_positions.append(gps_pos)

true_positions = np.array(true_positions)
imu_positions = np.array(imu_positions)
gps_positions = np.array(gps_positions)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: Trajectories
ax1 = axes[0]
ax1.plot(true_positions[:, 0], true_positions[:, 1], 
         'g-', linewidth=3, label='True Trajectory', zorder=5)
ax1.plot(imu_positions[:, 0], imu_positions[:, 1], 
         'r-', linewidth=2, alpha=0.7, label='IMU Dead Reckoning', zorder=4)
ax1.scatter(gps_positions[:, 0], gps_positions[:, 1], 
            c='blue', s=50, marker='x', label='GPS (1 Hz)', zorder=3)

ax1.set_xlabel('X Position (m)', fontsize=12)
ax1.set_ylabel('Y Position (m)', fontsize=12)
ax1.set_title('IMU Dead Reckoning vs GPS', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.axis('equal')

# Mark start and end
ax1.scatter([0], [0], c='green', s=200, marker='o', edgecolors='black', 
            linewidths=2, label='Start', zorder=10)
ax1.scatter([true_positions[-1, 0]], [true_positions[-1, 1]], 
            c='red', s=200, marker='s', edgecolors='black', 
            linewidths=2, label='End', zorder=10)

# Right: Error over time
ax2 = axes[1]

imu_error = np.linalg.norm(imu_positions - true_positions, axis=1)
time_steps = np.arange(len(imu_error)) * dt

ax2.plot(time_steps, imu_error, 'r-', linewidth=2, label='IMU Position Error')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Position Error (m)', fontsize=12)
ax2.set_title('IMU Drift Over Time', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)

# Add drift rate annotation
final_error = imu_error[-1]
drift_rate = final_error / T
ax2.text(0.5, 0.95, f'Final error: {final_error:.2f} m\nDrift rate: {drift_rate:.3f} m/s',
         transform=ax2.transAxes, fontsize=11, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

print("=" * 70)
print("IMU DEAD RECKONING ANALYSIS")
print("=" * 70)
print(f"Simulation time: {T} seconds")
print(f"IMU update rate: {1/dt:.0f} Hz")
print(f"GPS update rate: {gps_update_rate} Hz")
print(f"\nIMU Sensor Parameters:")
print(f"  Accelerometer noise: {imu_dr.acc_noise_std:.3f} m/s²")
print(f"  Accelerometer bias: {imu_dr.acc_bias}")
print(f"  Gyroscope noise: {imu_dr.gyro_noise_std:.3f} rad/s")
print(f"  Gyroscope bias: {imu_dr.gyro_bias:.4f} rad/s")
print(f"\nPosition Error:")
print(f"  Initial: {imu_error[0]:.2f} m")
print(f"  Final: {imu_error[-1]:.2f} m")
print(f"  Mean: {np.mean(imu_error):.2f} m")
print(f"  Drift rate: {drift_rate:.3f} m/s")
print(f"\nConclusion: IMU alone drifts unboundedly. GPS fusion required!")
print("=" * 70)

## 2. Inertial Measurement Units (IMU) and Dead Reckoning

An **IMU** measures acceleration and angular velocity using accelerometers and gyroscopes. Combined with GPS, IMUs provide high-frequency pose updates.

### IMU Components

**Accelerometer** (3-axis):
- Measures specific force (acceleration - gravity)
- Output: $\mathbf{a} = [a_x, a_y, a_z]^T$ in m/s²
- Noise: Bias drift, white noise

**Gyroscope** (3-axis):
- Measures angular velocity
- Output: $\boldsymbol{\omega} = [\omega_x, \omega_y, \omega_z]^T$ in rad/s
- Noise: Bias drift (major issue for integration)

**Magnetometer** (optional):
- Measures magnetic field for heading
- Subject to magnetic interference (motors, metal)

---

### Dead Reckoning with IMU

**Integrate acceleration to get velocity, then position**:

$$\mathbf{v}_{t+1} = \mathbf{v}_t + \mathbf{a}_t \cdot \Delta t$$

$$\mathbf{p}_{t+1} = \mathbf{p}_t + \mathbf{v}_t \cdot \Delta t + \frac{1}{2} \mathbf{a}_t \cdot \Delta t^2$$

**Integrate angular velocity to get orientation**:

$$\theta_{t+1} = \theta_t + \omega_t \cdot \Delta t$$

---

### Error Accumulation (Drift)

**Problem**: Integration amplifies noise → unbounded error growth.

**Acceleration noise** → Velocity error grows linearly → Position error grows quadratically.

**Example**: 
- Accelerometer bias: 0.01 m/s²
- After 10 seconds: Position error ≈ 0.5 meters
- After 60 seconds: Position error ≈ 18 meters

**Gyroscope drift**:
- Bias: 0.1°/s
- After 60 seconds: Heading error ≈ 6°

**Solution**: Fuse with GPS (Kalman Filter) to bound drift.

---

### IMU-GPS Fusion

**Complementary strengths**:
- **GPS**: Low-rate (1-10 Hz), bounded error, global
- **IMU**: High-rate (100-1000 Hz), drift, local

**Extended Kalman Filter (EKF)**:
- **Prediction**: Use IMU to propagate state at high rate
- **Update**: Use GPS to correct drift (when available)

**State vector**:
$$\mathbf{x} = [x, y, v_x, v_y, \theta, \omega, a_x, a_y, b_{gyro}, b_{acc}]^T$$

Includes position, velocity, orientation, angular velocity, acceleration, and sensor biases.

---

### IMU Types and Performance

| IMU Grade | Gyro Bias | Acc Bias | Cost | Application |
|-----------|-----------|----------|------|-------------|
| **Consumer** | 10-100°/hr | 0.01 m/s² | $10-100 | Smartphones, drones |
| **Tactical** | 1-10°/hr | 0.001 m/s² | $1k-10k | Automotive, robotics |
| **Navigation** | <1°/hr | <0.0001 m/s² | $10k-100k | Aircraft, submarines |

**Automotive AVs** typically use **Tactical-grade** IMUs (e.g., Xsens, VectorNav, Novatel).

---

### Wheel Odometry

**Alternative/complement to IMU**: Measure wheel rotations.

**Advantages**:
- ✅ No drift in straight-line motion (on flat ground)
- ✅ Accurate speed measurement
- ✅ Low cost (use existing wheel speed sensors)

**Disadvantages**:
- ❌ Wheel slip (rain, ice, acceleration)
- ❌ Tire wear affects calibration
- ❌ Doesn't measure lateral motion

**Fusion**: Combine GPS + IMU + Wheel Odometry for best results.

## Exercises

### Exercise 1: Particle Filter for Kidnapped Robot Problem

Implement a particle filter that can recover when the robot is "kidnapped" (teleported to an unknown location).

**Tasks:**
1. Modify the particle filter to occasionally inject random particles
2. Simulate kidnapping the robot at step 5
3. Show how the filter recovers
4. Compare recovery time with different numbers of random particles injected

**Hint:** Add uniform random particles when effective sample size drops below a threshold.

# Exercise 3 - Starter code

def create_multi_lane_map():
    """Create HD map with 3 lanes"""
    # TODO: Define lane centerlines
    # Lane 1: y = 0
    # Lane 2: y = 3.5
    # Lane 3: y = 7.0
    pass

def lane_level_localization():
    """Determine which lane the vehicle is in"""
    # TODO:
    # 1. Get GPS position (noisy)
    # 2. Detect lane markings (lateral offset)
    # 3. Fuse to determine lane ID
    # 4. Track lane changes
    pass

# Your implementation here

## References and Additional Resources

### Core Textbooks

1. **Probabilistic Robotics** by Thrun, Burgard, and Fox (2005)
   - Chapter 8: Mobile Robot Localization (Monte Carlo Localization)
   - Chapter 10: SLAM
   - Chapter 13: FastSLAM

2. **State Estimation for Robotics** by Barfoot (2017)
   - Chapters on SLAM and mapping

3. **Autonomous Driving** by Yurtsever et al. (2020)
   - Survey on localization methods for autonomous vehicles

### Key Papers

1. **Monte Carlo Localization**
   - Dellaert et al. (1999) - "Monte Carlo Localization for Mobile Robots"
   - Fox et al. (1999) - "Monte Carlo Localization: Efficient Position Estimation"

2. **SLAM**
   - Dissanayake et al. (2001) - "A Solution to the SLAM Problem"
   - Durrant-Whyte & Bailey (2006) - "Simultaneous Localization and Mapping: Part I & II"
   - Cadena et al. (2016) - "Past, Present, and Future of SLAM" (Survey)

3. **Visual SLAM**
   - Mur-Artal & Tardós (2017) - "ORB-SLAM2: an Open-Source SLAM System"
   - Campos et al. (2021) - "ORB-SLAM3"

4. **HD Maps**
   - Seif & Hu (2016) - "Autonomous Driving in the iCity - HD Maps as a Key Challenge"
   - Liu et al. (2020) - "High Definition Maps for Autonomous Driving"

### Software & Libraries

1. **SLAM Libraries**
   - **g2o**: General Graph Optimization - https://github.com/RainerKuemmerle/g2o
   - **GTSAM**: Georgia Tech Smoothing and Mapping - https://github.com/borglab/gtsam
   - **ORB-SLAM3**: Visual SLAM - https://github.com/UZ-SLAMLab/ORB_SLAM3
   - **Cartographer**: Google's SLAM - https://github.com/cartographer-project/cartographer

2. **Particle Filter**
   - **particle-filter-tutorial**: Python implementations
   - ROS packages: amcl (Adaptive Monte Carlo Localization)

3. **HD Maps**
   - **Lanelet2**: https://github.com/fzi-forschungszentrum-informatik/Lanelet2
   - **OpenDRIVE**: Industry standard format
   - **Apollo HD Map**: Baidu's open-source HD map module

### Online Resources

1. **Cyrill Stachniss YouTube Lectures**
   - Mobile Sensing and Robotics course
   - Excellent SLAM and localization lectures

2. **Self-Driving Cars Specialization (Coursera)**
   - University of Toronto
   - Modules on localization and mapping

3. **Udacity Self-Driving Car Nanodegree**
   - Particle filter project
   - Kidnapped vehicle project

### Datasets

1. **KITTI Dataset**: Autonomous driving benchmark
2. **nuScenes**: Full sensor suite with HD maps
3. **Waymo Open Dataset**: Large-scale AV dataset
4. **Oxford RobotCar**: Long-term autonomy dataset

### Industry Standards

- **ISO 19157**: Geographic Information - Data Quality
- **OpenDRIVE**: Road network description
- **NDS (Navigation Data Standard)**: Automotive navigation
- **Lanelet/Lanelet2**: Autonomous driving maps

### Applications in Autonomous Vehicles

1. **Localization Stack**
   - GPS/GNSS: Coarse global position
   - IMU: High-rate pose updates
   - Wheel odometry: Dead reckoning
   - LIDAR: Map matching and obstacle detection
   - Camera: Lane detection and visual odometry

2. **Map Update Strategies**
   - Crowdsourced mapping (fleet learning)
   - Real-time dynamic layer updates
   - Cloud-based map management

3. **Challenges**
   - Urban canyons (GPS multipath)
   - Tunnels (no GPS)
   - Dynamic environments
   - Map scalability and storage

---

## Summary

This notebook covered localization and mapping for autonomous vehicles:

**Key Concepts:**
- GPS/GNSS principles and error sources
- Particle Filters for global localization
- SLAM (Simultaneous Localization and Mapping)
- HD Maps for precise localization

**Practical Skills:**
- Implementing particle filters with resampling
- Building EKF-SLAM systems
- Map-matching for improved accuracy
- Multi-sensor fusion strategies

**Next Steps:**
1. Implement a full sensor fusion system
2. Explore Graph-SLAM and optimization-based approaches
3. Study Visual-Inertial Odometry (VIO)
4. Learn about semantic SLAM and dynamic environments